In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import datetime
from pathlib import Path
import glob

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf


%matplotlib inline


Bad key "text.kerning_factor" on line 4 in
C:\Users\Andy L\anaconda3\envs\dev\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution
C:\Users\Andy L\anaconda3\envs\dev\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Andy L\anaconda3\envs\dev\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Andy L\anaconda3\envs\dev\lib\site-packages\

In [2]:
# Import Dataset
# file_path_21 = Path("Resources/batting_stats_2021.csv")
# file_path_20 = Path("Resources/batting_stats_2020.csv")
# file_path_19 = Path("Resources/batting_stats_2019.csv")
# file_path_18 = Path("Resources/batting_stats_2018.csv")
# file_path_17 = Path("Resources/batting_stats_2017.csv")
# file_path_16 = Path("Resources/batting_stats_2016.csv")
# file_path_15 = Path("Resources/batting_stats_2015.csv")
# file_path_14 = Path("Resources/batting_stats_2014.csv")
# file_path_13 = Path("Resources/batting_stats_2013.csv")
# file_path_12 = Path("Resources/batting_stats_2012.csv")
# file_path_11 = Path("Resources/batting_stats_2011.csv")
# file_path_10 = Path("Resources/batting_stats_2010.csv")

# Read into Dataset
# stats_21_df = pd.read_csv(file_path_21, index_col = 0)
# stats_20_df = pd.read_csv(file_path_20, index_col = 0)
# stats_19_df = pd.read_csv(file_path_19, index_col = 0)
# stats_18_df = pd.read_csv(file_path_18, index_col = 0)
# stats_17_df = pd.read_csv(file_path_17, index_col = 0)
# stats_16_df = pd.read_csv(file_path_16, index_col = 0)
# stats_15_df = pd.read_csv(file_path_15, index_col = 0)
# stats_14_df = pd.read_csv(file_path_14, index_col = 0)
# stats_13_df = pd.read_csv(file_path_13, index_col = 0)
# stats_12_df = pd.read_csv(file_path_12, index_col = 0)
# stats_11_df = pd.read_csv(file_path_11, index_col = 0)
# stats_10_df = pd.read_csv(file_path_10, index_col = 0)

# stats_19_df.head(20)

In [3]:
# Import dataset
path = "Resources/batting_stats_*.csv"
all_files = glob.glob(path)
all_files

# Read & combine into dataframe
data = []

for file in all_files:
    df = pd.read_csv(file, index_col=None, header=0)    
    year = file[-8:-4]
    df['Year'] = year
    data.append(df)

stats_df = pd.concat(data, axis=0, ignore_index=True)
stats_df = stats_df[['Year', 'Player', 'Team', 'Pos', 'Age', 'G', 'AB', 'R', 'H', '2B', '3B', 'HR','RBI', 
             'SB', 'CS', 'BB', 'SO', 'SH', 'SF', 'HBP', 'AVG', 'OBP', 'SLG','OPS']]

# Drop player with 0 OPS
stats_df = stats_df[stats_df.OPS !=0].set_index('Player')

stats_df.head(100)

,Year,Team,Pos,Age,G,AB,R,H,2B,3B,...,CS,BB,SO,SH,SF,HBP,AVG,OBP,SLG,OPS
Player,,,,,,,,,,,,,,,,,,,,,
Ichiro Suzuki,2010,SEA,OF,47,162,680,74,214,30,3,...,9,45,86,3,1,3,0.315,0.359,0.394,0.753
Derek Jeter,2010,NYY,SS,47,157,663,111,179,30,3,...,5,63,106,1,3,9,0.270,0.340,0.370,0.710
Michael Young,2010,TEX,3B,44,157,656,99,186,36,3,...,2,50,115,0,11,1,0.284,0.330,0.444,0.774
Juan Pierre,2010,CWS,OF,43,160,651,96,179,18,3,...,18,45,47,15,2,21,0.275,0.341,0.316,0.657
Rickie Weeks,2010,MIL,DH,38,160,651,112,175,32,4,...,4,76,184,0,2,25,0.269,0.366,0.464,0.830
Marco Scutaro,2010,BOS,2B,45,150,632,92,174,38,0,...,4,53,71,4,3,3,0.275,0.333,0.388,0.721
Nick Markakis,2010,BAL,OF,37,160,629,79,187,45,3,...,2,73,93,0,5,2,0.297,0.370,0.436,0.806
Denard Span,2010,MIN,OF,37,153,629,85,166,24,10,...,4,60,74,10,2,4,0.264,0.331,0.348,0.679
Brandon Phillips,2010,CIN,2B,40,155,626,100,172,33,5,...,12,46,83,6,1,8,0.275,0.332,0.430,0.762


In [4]:
# Creating X & y variables
X = stats_df.iloc[:, 4:-2]
y = stats_df["OPS"].values

In [5]:
# Creating training, validation, and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

X_train.shape

(5914, 17)

In [6]:
# Scale the data
scaler = StandardScaler().fit(X)
X = scaler.transform(X)

y = y.reshape(-1, 1)
scaler_y = StandardScaler().fit(y)
y = scaler_y.transform(y)

In [7]:
y_train.shape

(5914,)

In [8]:
#X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
#X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [9]:
# Model set-up
number_input_features = 77
hidden_nodes_layer1 = 34
hidden_nodes_layer2 = 5

In [10]:
# Define the LSTM RNN model
model = Sequential()

# Layer 1
model.add(
    Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Layer 2
model.add(Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
model.add(Dense(1, activation="sigmoid"))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [11]:
# Compile the model
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=[
        "accuracy",
        tf.keras.metrics.TruePositives(name="tp"),
        tf.keras.metrics.TrueNegatives(name="tn"),
        tf.keras.metrics.FalsePositives(name="fp"),
        tf.keras.metrics.FalseNegatives(name="fn"),
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall"),
        tf.keras.metrics.AUC(name="auc"),
    ],
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [12]:
# Training the model
batch_size = 1000
epochs = 50
training_history = model.fit(
    X_train,
    y_train,
    #validation_data=(X_val, y_val),
    epochs=epochs,
    batch_size=batch_size,
    verbose=1,
)

Epoch 1/50
5914/5914 [==============================] - 0s 24us/sample - loss: 21.3979 - acc: 0.0034 - tp: 5914.0000 - tn: 0.0000e+00 - fp: 0.0000e+00 - fn: 0.0000e+00 - precision: 1.0000 - recall: 1.0000 - auc: 0.0000e+00
Epoch 2/50
5914/5914 [==============================] - 0s 2us/sample - loss: 18.5577 - acc: 0.0034 - tp: 5914.0000 - tn: 0.0000e+00 - fp: 0.0000e+00 - fn: 0.0000e+00 - precision: 1.0000 - recall: 1.0000 - auc: 0.0000e+00
Epoch 3/50
5914/5914 [==============================] - 0s 2us/sample - loss: 15.8953 - acc: 0.0034 - tp: 5914.0000 - tn: 0.0000e+00 - fp: 0.0000e+00 - fn: 0.0000e+00 - precision: 1.0000 - recall: 1.0000 - auc: 0.0000e+00
Epoch 4/50
5914/5914 [==============================] - 0s 3us/sample - loss: 13.3533 - acc: 0.0034 - tp: 5914.0000 - tn: 0.0000e+00 - fp: 0.0000e+00 - fn: 0.0000e+00 - precision: 1.0000 - recall: 1.0000 - auc: 0.0000e+00
Epoch 5/50
5914/5914 [==============================] - 0s 3us/sample - loss: 10.8820 - acc: 0.0034 - tp: 5914.

In [13]:
# Make predictions using the testing data X_test
predicted = model.predict(X_test)

# Evaluate the model
model.evaluate(X_test, y_test)

1972/1972 [==============================] - 0s 46us/sample - loss: 0.5720 - acc: 0.0056 - tp: 1968.0000 - tn: 0.0000e+00 - fp: 0.0000e+00 - fn: 4.0000 - precision: 1.0000 - recall: 0.9980 - auc: 0.0000e+00


[0.5720295413996095, 0.005578093, 1968.0, 0.0, 0.0, 4.0, 1.0, 0.9979716, 0.0]

In [14]:
predicted_OPS = scaler_y.inverse_transform(predicted)

In [15]:
moneyball = pd.DataFrame({
    "Real": y_test.ravel(),
    "Predicted": predicted_OPS.ravel()
    }, index = stats_df.index[-len(predicted_OPS): ])
moneyball.head(20)

,Real,Predicted
Player,,
Jacob Nottingham,0.950,0.816808
Taylor Motter,0.651,0.797225
Ildemaro Vargas,0.616,0.808865
Adam Rosales,0.614,0.771530
Brandon Barnes,0.661,0.789293
Anthony Alford,0.646,0.791086
Luis Sardinas,1.185,0.858469
Matt den Dekker,0.419,0.779953
Corban Joseph,0.486,0.793946
